In [14]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

import scipy.stats as ss
from scipy import linalg  # numpy linalg plus extras
import statsmodels.api as sm

import scipy.io as spio
import matlab.engine

ModuleNotFoundError: No module named 'matlab'

In [12]:
ddir='/data1/adac/process/'
fn='adac_tsg_segments_vector.mat'
print(ddir + fn)
tsgseg = spio.loadmat(ddir + fn)
print(tsgseg)

/data1/adac/process/adac_tsg_segments_vector.mat
{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Mon Dec 28 17:32:36 2020', '__version__': '1.0', '__globals__': [], 'tsgseg_vector': array([[(array([[733594.26736111, 733594.2712576 , 733594.27509782, ...,
        737307.50498681, 737307.50930638,             nan]]), array([[ 2.,  2.,  2., ..., 26., 26., nan]]), array([[-169.13850403, -169.11563116, -169.09277179, ..., -125.13946334,
        -125.1442332 ,           nan]]), array([[-14.60849953, -14.61245001, -14.61647491, ...,  44.60753644,
         44.5853127 ,          nan]]), array([[        nan,         nan,         nan, ..., 14.17456987,
        14.07999992,         nan]]), array([[35.16500092, 35.1504066 , 35.13692358, ..., 32.2518098 ,
        32.27000046,         nan]]), array([[  1.,   1.,   1., ..., 332., 332.,  nan]]), array([[44178256.        , 44178257.1225972 , 44178258.23073516, ...,
        42719021.18101028, 42719027.40119305,               nan]]), 